In [19]:
import random
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from datetime import datetime

# Initialize geolocator and constants
geolocator = Nominatim(user_agent="advanced_flight_calculator")
average_flight_speed = 500  # mph
fuel_burn_per_mile = 3.5  # gallons per mile
fuel_price_per_gallon = 3.00  # USD per gallon
passenger_count = 150  # narrow-body aircraft capacity
airport_charges_base = 2500  # base airport/landing fee
economy_base_fare_direct = 250  # Base fare for direct flights

# Define holiday peak seasons
peak_season = [("12-15", "01-05"), ("07-01", "07-10"), ("11-20", "11-30")]

# Function to get latitude and longitude of a city/airport
def get_location(city):
    try:
        location = geolocator.geocode(city)
        if location:
            return (location.latitude, location.longitude)
        else:
            raise ValueError(f"Location not found for {city}.")
    except Exception as e:
        raise ValueError(f"Error getting location for {city}: {e}")

# Function to calculate distance between two cities
def calculate_distance(city1, city2):
    loc1 = get_location(city1)
    loc2 = get_location(city2)
    return geodesic(loc1, loc2).miles if loc1 and loc2 else None

# Adjust fare based on date proximity and holiday
def calculate_fare(base_fare, travel_date):
    current_date = datetime.now()
    days_until_travel = (travel_date - current_date).days
    peak_multiplier = 1.2 if any(start <= travel_date.strftime("%m-%d") <= end for start, end in peak_season) else 1.0
    booking_window_multiplier = 1.5 if days_until_travel <= 5 else 1.0
    adjusted_fare = base_fare * peak_multiplier * booking_window_multiplier
    fare_cap = 350  # Direct flight fare cap
    return min(adjusted_fare, fare_cap)

# Direct flight details
def direct_flight_details(start_city, end_city, travel_date):
    distance = calculate_distance(start_city, end_city)
    if not distance:
        raise ValueError("Unable to calculate distance for the provided cities.")

    travel_time = distance / average_flight_speed
    fuel_cost = distance * fuel_burn_per_mile * fuel_price_per_gallon
    airport_charges = airport_charges_base + 0.1 * distance
    economy_fare = calculate_fare(economy_base_fare_direct, travel_date)
    first_class_fare = economy_fare * 1.5

    total_revenue = economy_fare * passenger_count
    profit = total_revenue - (fuel_cost + airport_charges)

    print("\nDirect Flight Details:")
    print(f"  Start City: {start_city}")
    print(f"  End City: {end_city}")
    print(f"  Distance: {distance:.2f} miles")
    print(f"  Travel Time: {travel_time:.2f} hours")
    print(f"  Fuel Cost: ${fuel_cost:.2f}")
    print(f"  Airport Charges: ${airport_charges:.2f}")
    print(f"  Economy Fare per Passenger: ${economy_fare:.2f}")
    print(f"  First Class Fare per Passenger: ${first_class_fare:.2f}")
    print(f"  Total Revenue: ${total_revenue:.2f}")
    print(f"  Profit: ${profit:.2f}")

    return {"economy_fare": economy_fare, "first_class_fare": first_class_fare}

# Example usage
start_city = input("Enter the start city: ").strip()
end_city = input("Enter the end city: ").strip()
travel_date_input = input("Enter the travel date (YYYY-MM-DD): ").strip()
travel_date = datetime.strptime(travel_date_input, "%Y-%m-%d")

direct_flight_fares = direct_flight_details(start_city, end_city, travel_date)


Enter the start city: chicago
Enter the end city: dallas
Enter the travel date (YYYY-MM-DD): 2024-11-25

Direct Flight Details:
  Start City: chicago
  End City: dallas
  Distance: 804.53 miles
  Travel Time: 1.61 hours
  Fuel Cost: $8447.55
  Airport Charges: $2580.45
  Economy Fare per Passenger: $350.00
  First Class Fare per Passenger: $525.00
  Total Revenue: $52500.00
  Profit: $41472.00


In [20]:
import random
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from datetime import datetime

# Initialize geolocator and constants
geolocator = Nominatim(user_agent="advanced_flight_calculator", timeout=5)
average_flight_speed = 500  # mph
fuel_burn_per_mile = 3.5  # gallons per mile
fuel_price_per_gallon = 3.00  # USD per gallon
passenger_count = 150  # narrow-body aircraft capacity
airport_charges_base = 2500  # base airport/landing fee
minimum_distance_between_cities = 50  # Minimum distance between cities for valid layover
min_layover_time = 2  # Minimum layover time in hours
max_layover_time = 5  # Maximum layover time in hours

# Define holiday peak seasons
peak_season = [("12-15", "01-05"), ("07-01", "07-10"), ("11-20", "11-30")]

# Function to get latitude and longitude of a city/airport
def get_location(city):
    try:
        location = geolocator.geocode(city)
        if location:
            return (location.latitude, location.longitude)
        else:
            raise ValueError(f"Location not found for {city}.")
    except Exception as e:
        raise ValueError(f"Error getting location for {city}: {e}")

# Function to calculate distance between two cities
def calculate_distance(city1, city2):
    try:
        loc1 = get_location(city1)
        loc2 = get_location(city2)
        return geodesic(loc1, loc2).miles if loc1 and loc2 else None
    except ValueError as e:
        raise ValueError(f"Distance calculation failed: {e}")

# Calculate economy fare dynamically based on distance
def calculate_economy_fare(distance, travel_date, fuel_cost, airport_charges, time_ratio):
    base_fare = (fuel_cost + airport_charges) / passenger_count
    distance_multiplier = distance * 0.05  # Per-mile fare adjustment
    total_fare = base_fare + distance_multiplier

    # Adjust for season, booking window, and time ratio
    current_date = datetime.now()
    days_until_travel = (travel_date - current_date).days
    peak_multiplier = 1.2 if any(start <= travel_date.strftime("%m-%d") <= end for start, end in peak_season) else 1.0
    booking_window_multiplier = 1.3 if days_until_travel <= 5 else 1.0
    time_discount = max(1 - (time_ratio - 1) * 0.1, 0.7)  # Discount based on time ratio; cap at 30% discount

    return total_fare * peak_multiplier * booking_window_multiplier * time_discount

# Adjust fares for connecting flights to ensure they are lower than direct flight fares
def adjust_connecting_fares(direct_economy, direct_first_class, economy_fare, time_ratio):
    # Ensure connecting economy fare is at least 20% cheaper than direct
    adjusted_economy_fare = min(economy_fare, direct_economy * 0.8)

    # Calculate first-class fare as a multiplier of economy, capped below direct first-class fare
    first_class_multiplier = 1.3  # Keep a smaller multiplier for realism
    adjusted_first_class_fare = min(adjusted_economy_fare * first_class_multiplier, direct_first_class * 0.8)

    # Apply additional discount if travel time is significantly higher
    if time_ratio > 1.2:  # If connecting travel time exceeds direct time by 20%
        adjusted_economy_fare *= 0.9  # Additional 10% discount
        adjusted_first_class_fare *= 0.9

    return adjusted_economy_fare, adjusted_first_class_fare

# Connecting flight details with controlled layover fare and full financials
def connecting_flight_details(start_city, end_city, travel_date, direct_flight_fares, direct_travel_time):
    # Retrieve direct flight fares
    direct_economy_fare = direct_flight_fares.get("economy_fare", 9999)
    direct_first_class_fare = direct_flight_fares.get("first_class_fare", 9999)

    # Get locations of start and end cities
    start_location = get_location(start_city)
    end_location = get_location(end_city)

    if not start_location or not end_location:
        raise ValueError("Could not retrieve location for start or end city.")

    # Define Sunbelt hub airports for connecting flights
    sunbelt_airports = [
        "Dallas/Fort Worth International Airport (DFW)",
        "Charlotte Douglas International Airport (CLT)",
        "Phoenix Sky Harbor International Airport (PHX)",
        "Miami International Airport (MIA)"
    ]

    # Filter layover options based on geographic proximity
    valid_layover_options = []
    for layover_city in sunbelt_airports:
        layover_location = get_location(layover_city)
        if layover_location:
            distance_from_start = geodesic(start_location, layover_location).miles
            distance_from_end = geodesic(end_location, layover_location).miles
            if distance_from_start > minimum_distance_between_cities and distance_from_end > minimum_distance_between_cities:
                valid_layover_options.append(layover_city)

    if not valid_layover_options:
        raise ValueError("No valid layover options available.")

    for layover_city in valid_layover_options[:2]:  # Limit to two options
        distance_leg1 = calculate_distance(start_city, layover_city)
        distance_leg2 = calculate_distance(layover_city, end_city)
        if distance_leg1 and distance_leg2:
            fuel_cost_leg1 = distance_leg1 * fuel_burn_per_mile * fuel_price_per_gallon
            fuel_cost_leg2 = distance_leg2 * fuel_burn_per_mile * fuel_price_per_gallon
            airport_charges_leg1 = airport_charges_base + 0.1 * distance_leg1
            airport_charges_leg2 = airport_charges_base + 0.1 * distance_leg2

            total_fuel_cost = fuel_cost_leg1 + fuel_cost_leg2
            total_airport_charges = airport_charges_leg1 + airport_charges_leg2
            total_distance = distance_leg1 + distance_leg2

            # Calculate total travel time
            layover_time = random.uniform(min_layover_time, max_layover_time)
            total_travel_time = (distance_leg1 / average_flight_speed) + (distance_leg2 / average_flight_speed) + layover_time

            # Calculate time ratio
            time_ratio = total_travel_time / direct_travel_time

            # Calculate economy fare dynamically
            economy_fare = calculate_economy_fare(total_distance, travel_date, total_fuel_cost, total_airport_charges, time_ratio)

            # Ensure fares are adjusted for profitability and undercut direct flight fares
            economy_fare, first_class_fare = adjust_connecting_fares(
                direct_economy_fare, direct_first_class_fare, economy_fare, time_ratio
            )

            total_revenue = economy_fare * passenger_count
            profit = total_revenue - (total_fuel_cost + total_airport_charges)

            print(f"\nConnecting Flight Option via {layover_city}")
            print(f"Path: {start_city} → {layover_city} → {end_city}")
            print(f"Total Distance: {total_distance:.2f} miles")
            print(f"Total Travel Time (including layover): {total_travel_time:.2f} hours")
            print(f"Fuel Cost: ${total_fuel_cost:.2f}")
            print(f"Airport Charges: ${total_airport_charges:.2f}")
            print(f"Economy Fare per Passenger: ${economy_fare:.2f}")
            print(f"First Class Fare per Passenger: ${first_class_fare:.2f}")
            print(f"Total Revenue: ${total_revenue:.2f}")
            print(f"Profit: ${profit:.2f}")

# Example usage
start_city = input("Enter the start city: ").strip()
end_city = input("Enter the end city: ").strip()
travel_date_input = input("Enter the travel date (YYYY-MM-DD): ").strip()
travel_date = datetime.strptime(travel_date_input, "%Y-%m-%d")

# Example direct flight fares and travel time (replace with actual calculation)
direct_flight_fares = {"economy_fare": 350, "first_class_fare": 600}
direct_travel_time = 4.0  # Replace with actual direct travel time in hours

# Generate connecting flight details
connecting_flight_details(start_city, end_city, travel_date, direct_flight_fares, direct_travel_time)


Enter the start city: chicago
Enter the end city: dallas
Enter the travel date (YYYY-MM-DD): 2024-11-25

Connecting Flight Option via Charlotte Douglas International Airport (CLT)
Path: chicago → Charlotte Douglas International Airport (CLT) → dallas
Total Distance: 1508.88 miles
Total Travel Time (including layover): 6.97 hours
Fuel Cost: $15843.26
Airport Charges: $5150.89
Economy Fare per Passenger: $252.00
First Class Fare per Passenger: $327.60
Total Revenue: $37800.00
Profit: $16805.85

Connecting Flight Option via Phoenix Sky Harbor International Airport (PHX)
Path: chicago → Phoenix Sky Harbor International Airport (PHX) → dallas
Total Distance: 2334.18 miles
Total Travel Time (including layover): 9.30 hours
Fuel Cost: $24508.90
Airport Charges: $5233.42
Economy Fare per Passenger: $252.00
First Class Fare per Passenger: $327.60
Total Revenue: $37800.00
Profit: $8057.68
